In [1]:
import numpy as np
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
# import gensim.downloader as api
# import pickle
from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm
tqdm.pandas()
import re

In [2]:
html_tag = {'</p>', '<br>', '<p>', '</li>', '<li>', '</ul>', '<ul>', '</div>', '<div>',
                 '</h2>', '<h2>', '</h1>', '<h1>', '<h3>', '</h3>', '</h4>', '<h4>', '<h5>', '</h5>',
                 '</body>', '<body>', '</html>', '<html>', '<form>', '</title>', '<title>',
              '</layer>', '<layer>', '</iframe>', '<iframe>', '</form>', '</span>', '<span>',
              '<input>', '</input>', '</comment>', '<comment>', '</textarea>', '<textarea>',
              '</ilayer>', '<ilayer>', '<head>', '</head>'}

html_chars = '|'.join(map(re.escape, html_tag))

removed_chars = {'•', '♦'}

# removed_chars = {'•', '⁈', '⁉', '․', '‥', '…', '‧', '★', '☆', '☝', '☭', '♁',
#                  '♀', '♂', '☹', '☺', '☻', '♠', '♡', '♢', '♣', '♤', '—',
#                  '♥', '♦', '♧', '♩', '♪', '♫', '♬', '♛', '♚', '♜', '♞'}

In [3]:
sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation).union(removed_chars)

In [4]:
def preprocess_txt(line):
    spls = re.sub(r"https?://[^,\s]+,?", " ", line)
    spls = re.sub(r'\d+\s?', " ", spls).strip()
    spls = re.sub(r'(\b\w+)\s+\1', r'\1', spls)
    spls = re.sub(r'[^а-яА-Яё ]', ' ', spls)
    spls = " ".join(re.sub(html_chars, ' ', spls).split())
    spls = "".join(i if i not in exclude else ' ' for i in spls.strip()).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [6]:
question = None
written = False

with open("prepared_answers.txt", "w", encoding='utf-8') as fout:
    with open("J:/Storage_for_ML/Otvety.txt", "r", encoding='utf-8', errors="ignore") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---") or line == '\n':
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

C:\Users\Вадим\AppData\Local\Temp\ipykernel_20920\3283748491.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [8]:
i = 0
with open("J:/Storage_for_ML/Otvety.txt", "r", encoding='utf-8', errors="ignore") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        if spls:
            sentences.append(spls)
            i += 1
        if i > 150000:
            break

C:\Users\Вадим\AppData\Local\Temp\ipykernel_20920\2663025624.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [9]:
sentences = [i for i in sentences if len(i) > 2]
print(sentences[0])

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']


In [10]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [11]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [12]:
w2v_index = annoy.AnnoyIndex(300, 'angular')

In [13]:
ft_index = annoy.AnnoyIndex(300, 'angular')

In [14]:
index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding='utf-8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])

        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1

        if counter > 150000:
            break

C:\Users\Вадим\AppData\Local\Temp\ipykernel_20920\438557804.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

In [15]:
w2v_index.build(10)

True

In [16]:
ft_index.build(10)

True

In [17]:
TEXT = "какой город самый красивый"

In [18]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['говорят, что самые красотки живут в Ростове.. \n', 'я думаю в Москве девушки более ухоженные, а красивые они везде.. \n', 'В<br>XI веке некоторые монастырские школы, например школа в Беке, привлекала<br>учеников из разных мест. Кафедральные школы также были клерикальными, то<br>есть занимались обучением клириков, принадлежавших к религиозным орденам. В<br>XII веке некоторые из этих школ были знаменитыми центрами учености -<br>например, Шартрская и Парижская. <br><br>Исследования в области права и медицины сначала сосредотачивались в<br>Италии. В некоторых итальянских городах, например в Павии, существовали<br>городские школы права, но в XII веке всех их затмила Школа в Болонье. Здесь<br>в первой половине века выступал Ирнерий с лекциями и комментариями к<br>кодексу Юстиниана. И именно здесь камальдолийский монах Грациан создал<br>кодекс канонического права, который облегчил обучение квалифицированных<br>правоведов и юристов для нужд Церкви. Если говорить о медицинских<br>исследования

In [19]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['говорят, что самые красотки живут в Ростове.. \n', 'видс крыши Исакиевского собора. \n', 'Белгород самый чистый, Тамбов - уже не город, а свалка.. \n']


In [20]:
TEXT = "какой день недели"

In [21]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['Семидневная неделя как промежуточная единица измерения времени между сутками и месяцем возникла в Древнем Вавилоне. Отсюда она перешла к евреям, а затем к грекам и римлянам; от римлян она широко распространилась по всей Западной Европе. Семидневная неделя получила признание и у многих народов арабского Востока. Вавилоняне приписывали числу «семь» магическое значение, считая его «священным» . Такое почитание было связано с числом известных в то время планет (к которым относили также Солнце и Луну) .<br><br>Вероятно, происхождение семидневной недели как единицы измерения времени связано и с другой причиной — с изменением лунных фаз, повторяющихся через каждые 29,5 суток. Если учесть, что во время новолуния Луна не видна около 1,5 суток, то продолжительность видимости ее составит 28 суток, или четыре недели. И теперь период изменения вида Луны мы делим на четыре части, которые называем первой четвертью, полнолунием, последней четвертью и новолунием. Каждая четверть лунного месяца длится

In [22]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['Семидневная неделя как промежуточная единица измерения времени между сутками и месяцем возникла в Древнем Вавилоне. Отсюда она перешла к евреям, а затем к грекам и римлянам; от римлян она широко распространилась по всей Западной Европе. Семидневная неделя получила признание и у многих народов арабского Востока. Вавилоняне приписывали числу «семь» магическое значение, считая его «священным» . Такое почитание было связано с числом известных в то время планет (к которым относили также Солнце и Луну) .<br><br>Вероятно, происхождение семидневной недели как единицы измерения времени связано и с другой причиной — с изменением лунных фаз, повторяющихся через каждые 29,5 суток. Если учесть, что во время новолуния Луна не видна около 1,5 суток, то продолжительность видимости ее составит 28 суток, или четыре недели. И теперь период изменения вида Луны мы делим на четыре части, которые называем первой четвертью, полнолунием, последней четвертью и новолунием. Каждая четверть лунного месяца длится

In [23]:
TEXT = "как кошка с собакой"

In [24]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['Совершенно разные животные.<br>Собачки даже маленькие подчиняются хозяину,<br>кошечки же и коты же свободолюбивые и своенравные.. \n', 'может конечно. \n', 'котяра. \n']


In [25]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['котяра. \n', 'Совершенно разные животные.<br>Собачки даже маленькие подчиняются хозяину,<br>кошечки же и коты же свободолюбивые и своенравные.. \n', 'Бред.. \n']


In [26]:
TEXT = "как высадить дерево"

In [27]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['<br>А мне и тут хорошо ...я тут зависаю потихому!! Давай к нам!!!. \n', '<p>Ним, или Ниим, или Азадирахта индийская (лат.\xa0Azadirachta indica), также маргоза\xa0— древесное растение, вид рода Азадирахта (Azadirachta) семейства Мелиевые. Происходит из тропических и субтропических районов Пакистана, Бангладеш, Индии и Мьянмы. </p> <h2>Биологическое описание</h2> <p>Вечнозелёное дерево высотой 12—18\xa0м с раскидистой кроной. </p> <h2>Использование</h2> <p>В Индии растение известно как «деревенская аптека», «божественное дерево» и «панацея от всех болезней». Продукция, сделанная из него, в аюрведической медицине считается глистогонным, противогрибковым, противобактериальным, противовирусным, антидиабетическим и успокоительным средствами; также препятствует образованию перхоти. Ним известен также как природный очиститель крови. Ствол, корни и кора обладают вяжущими, общеукрепляющими, противомалярийными свойствами. В аюрведической медицине средства, приготовленные из этого дерева, рассм

In [28]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['<div> <p>Посадить дерево, построить дом и вырастить сына. Сегодня, дорогой читатель, мы затронем первый аспект этой священной мужской миссии. Ответим на вопрос о том, как посадить дерево. Итак:</p> <h2>Сажаем дерево</h2> <p>Существует даже такой праздник, который носит гордое название "День посадки деревьев". Жители многих стран осуществляют деревянные насаждения в окрестностях своего города, тем самым, заботясь о растениях и экологии Земли. В нашей стране этого праздника не существует, хотя наши ближайшие соседи (Украина) празднуют его каждую третью субботу апреля.</p> <p>Как правильно посадить дерево? Самое важное в посадке - это соблюдение жизненно необходимых правил. Вряд ли дерево вырастит, если вы просто выкопаете яму, засунете туда саженец и засыпаете землей.</p> <p>Прежде всего, необходимо понять, какое дерево вы хотите посадить.</p> <ul> <li>Покупка саженца должна проходить в крупном магазине или на садоводческой ферме, где можно получить полную консультацию о том или ином р

In [29]:
TEXT = "какая страна самая красивая"

In [30]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['Россия, Украина, Беларусь!. \n', 'Альберт!!!. \n', 'меня жена простила и вернулась! что может быть лучше? я самый счастливый человек на свете!!!. \n']


In [31]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['Россия, Украина, Беларусь!. \n', 'ТОП-15 самых некрасивых футболистов ЧМ-2010 <br> <br> 1. Франк Рибери – 2,98 (Франция) . <br> 2. Хеендри Томас – 3,00 (Гондурас) . <br> 3. Нванкво Кану – 3,05 (Нигерия) . <br> 4. Денис Каниса – 3,12 (Парагвай) . <br> 5. Ригобер Сонг – 3,13 (Камерун) . <br> 6. Скотт Чипперфилд – 3,15 (Австралия) . <br> 7. Зденко Штрба – 3,21 (Словакия) . <br> 8. Роланду да Фонсека (Роландо) – 3,22 (Португалия) . <br> 9. Авраам Пападопулос – 3,23 (Греция) . <br> 10. Людовик Маньен – 3,25 (Швейцария) . <br> 11. Райан Нелсен – 3,26 (Новая Зеландия) . <br> 12. Исмаэль Тиоте – 3,27 (Кот-д’Ивуар) . <br> 13. Салли Мунтари – 3,28 (Гана) . <br> 14. Юити Комано – 3,32 (Япония) . <br> 15. Ким Кум Иль – 3,34 (КНДР) . <br> <br> Читать полностью: http://www.interfax.by/article/67905<br>. \n', 'Танго в плену у двоих <br>Страстью взрываются звуки, <br>Чувств не скрывая своих, <br>Властвуют нежные руки. <br> <br>Чарами в бездну маня, <br>Выплеснет в райские кущи <br>Пламенный танец ог

In [32]:
TEXT = "как пить водку"

In [33]:
print(get_response(TEXT, w2v_index, modelW2V, index_map))

['Да и желания вроде нет!. \n', 'оу... и правда... чет залипла тут... а кофе то? (((.... \n', 'Слабаки пьют кофе. я пью крепкий чай!)). \n']


In [34]:
print(get_response(TEXT, ft_index, modelFT, index_map))

['Да и желания вроде нет!. \n', 'Нет. во ремя еды по немногу. \n', 'Слабаки пьют кофе. я пью крепкий чай!)). \n']
